In [2]:
# NOTES
# Versioning persists after deletion: e.g. I delete version 2, next version will be 3 regardless.
# Send request with json.dumps(data) to wrap json format into a string, otherwise it does not work
# Pausing takes some time and a journey version cannot be resumed unless it is in status "Paused"

# Edge Cases:
## Weights for A/B Test do not add up to 100%
## Status must remain the same if in ("Running", "Draft", "Paused")
## Journey should be ignored if "Stopped"

# Requirements:
## Save logs in file
## Send logs to Karam
## Run async since each journey update takes 5' approx.

In [106]:
import pandas as pd
import numpy as np
import requests
import re
import json
import time
from time import sleep
import datetime
from math import *
import sys
import os
import getpass
import functools
import warnings
warnings.filterwarnings("ignore")

import concurrent.futures
from multiprocessing import Process, current_process
import time

# Logging

In [107]:
import logging

logging.basicConfig(filename='ikea_journey_weights_update.log', level=logging.DEBUG,
                    format="%(asctime)s:%(levelname)s:%(message)s")

# Handle Errors

In [108]:
class RequestError(Exception):
    
    def __init__(self, response, message):
        self.response = response
        self.message = message
        super().__init__(message)

class StatusError(Exception):
    
    def __init__(self, status, message):
        self.status = status
        self.message = message
        super().__init__(message)

# Journey Builder API Credentials

In [109]:
# Get credentials (IKEA env)

user = getpass.getuser()  # e.g. Doej for John Doe
filepath = "Z:\Shared\EMEA\CPHEBT\Client Information\Ikea\API Keys\\" + user + "\credentials.json"
creds = json.load(open(filepath))
client_id = creds['users'][user]['clientId']
client_secret = creds['users'][user]['secret']
subdomain = "mcjxbybvhgwn9kgd4wfv-kz3b6cq"

In [110]:
# Get credentials (Sandbox env)
"""with open("config.json") as credentials:
    credentials = json.load(credentials)

client_id = credentials["client_id"]
client_secret = credentials["client_secret"]
subdomain = "mc42bdlx7mz5h4np2xxvhsb4scvq"
""""

SyntaxError: EOL while scanning string literal (<ipython-input-110-bc4125d48e3e>, line 8)

# Journey Builder Class

In [111]:
class JourneyBuilder:
    """This class is meant to represent specific journeys within Salesforce Journey Builder API."""
    
    def __init__(self, journey_name):

        self.journey_name = journey_name
        
        # Authentication
        self.client_id = client_id
        self.client_secret = client_secret
        self.auth_url = f"https://{subdomain}.auth.marketingcloudapis.com/v2/token"
        self.auth_headers = {"content-type": "application/json"}
        self.auth_payload = {
            "grant_type": "client_credentials",
            "client_id": self.client_id,
            "client_secret": self.client_secret
        }
        
        self.rest_url = f"https://{subdomain}.rest.marketingcloudapis.com"

        try:
            self.access_token, self.access_token_expiration, self.rest_headers = self.get_access_token()
            if self.access_token is None:
                raise Exception("Request for access token failed.")
        except Exception as e:
            print(e)

    def get_access_token(self):
        """Get access token; associated with the decorator below."""
        try:
            authentication_response = requests.post(
                url=self.auth_url, 
                data=json.dumps(self.auth_payload), 
                headers=self.auth_headers, 
                verify=False)
            authentication_response.raise_for_status()
        except Exception as e:
            print(e)
            return None
        else:
            access_token = authentication_response.json()["access_token"]
            access_token_expiration = authentication_response.json()["expires_in"]
            rest_headers = {"authorization": f"Bearer {access_token}"}
            return access_token, access_token_expiration, rest_headers
    
    class Decorators():
        @staticmethod
        def refresh_token(decorated):
            """Methods with this decorator will refresh the access token if it is expired before running."""
            def wrapper(api,*args,**kwargs):
                if time.time() > api.access_token_expiration:
                    api.access_token, api.access_token_expiration, api.rest_headers = api.get_access_token()
                return decorated(api,*args,**kwargs)
            return wrapper
    
    @Decorators.refresh_token
    def get_metadata(self, *args):
        """Get journey metadata such as id, key, name, version, status etc.."""
        
        # Get all journeys for user
        response = requests.get(
            url=f"{self.rest_url}/interaction/v1/interactions/",
            headers=self.rest_headers,
            verify=False
        )
        # Raise exception if status not in 200s
        pattern = re.compile("20[0-9]")
        match = re.findall(pattern, str(response.status_code))
        if not match:
            raise RequestError(
                response, "get_metadata: Response not in 200s.")
                    
        # Return metadata for journey = journey_name
        return [[journey[arg] for journey in json.loads(response.content)["items"]
                if journey["name"] == self.journey_name][0] for arg in args]
    
    @Decorators.refresh_token
    def change_status(self, action):
        """Change journey status with an action: Pause, Resume, Stop."""
        
        journey_id, journey_version = self.get_metadata("id", "version")
        
        # The API needs to know where it has to perform the action: journey id + version.
        response = requests.post(
            url=f"{self.rest_url}/interaction/v1/interactions/{action}/{journey_id}?versionNumber={journey_version}",
            headers=self.rest_headers,
            verify=False
        )
        # Raise exception if status not in 200s
        pattern = re.compile("20[0-9]")
        match = re.findall(pattern, str(response.status_code))
        if not match:
            raise RequestError(
                response, f"change_status: Response not in 200s.")
    
    @Decorators.refresh_token
    def get_activities(self):
        """Get all journey activities with the journey id."""
        
        journey_id = self.get_metadata("id")[0]

        # Request gets all activities within journey based on id
        response = requests.get(
            url=f"{self.rest_url}/interaction/v1/interactions/{journey_id}",
            headers=self.rest_headers,
            verify=False
        )
        # Raise exception if status not in 200s
        pattern = re.compile("20[0-9]")
        match = re.findall(pattern, str(response.status_code))
        if not match:
            raise RequestError(
                response, "get_activities: Response not in 200s.")   
        return json.loads(response.content)
    
    @Decorators.refresh_token
    def update_version(self, journey_version_updated):
        """Update journey version with updated activities. Status cannot be 'Published'."""
        response = requests.put(
            url=f"{self.rest_url}/interaction/v1/interactions",
            headers=self.rest_headers,
            data=json.dumps(journey_version_updated),
            verify=False
        )
        # Raise exception if status not in 200s
        pattern = re.compile("20[0-9]")
        match = re.findall(pattern, str(response.status_code))
        if not match:
            raise RequestError(
                response, f"update_journey_version: Response not in 200s.")
            
    def __str__(self):
        
        return f"Journey Name = {self.journey_name}"

In [112]:
"""
# create_version method in case I need it at some point.
    
    @Decorators.refreshToken
    def create_version(self, journey_id, journey_version, new_perc):
        # Update journey version
        journey_activities = self.get_activities_by_id(journey_id)
        new_version_journey_activities = self.update_journey_perc(journey_activities, new_perc, version="new")
        # Create new jounrey version
        r = requests.post(
            url=f'{self.rest_url}/interaction/v1/interactions',
            headers=self.rest_headers,
            data=json.dumps(new_version_journey_activities),
            verify=False
        )
        logging.info(f"Journey: {journey_id, journey_version+1} CREATED SUCCESSFULLY!")
        # Stop old version
        self.change_status(journey_id, journey_version, "stop")
        logging.info(f"Journey: {journey_id, journey_version} STOPPED SUCCESSFULLY!")
        """

'\n# create_version method in case I need it at some point.\n    \n    @Decorators.refreshToken\n    def create_version(self, journey_id, journey_version, new_perc):\n        # Update journey version\n        journey_activities = self.get_activities_by_id(journey_id)\n        new_version_journey_activities = self.update_journey_perc(journey_activities, new_perc, version="new")\n        # Create new jounrey version\n        r = requests.post(\n            url=f\'{self.rest_url}/interaction/v1/interactions\',\n            headers=self.rest_headers,\n            data=json.dumps(new_version_journey_activities),\n            verify=False\n        )\n        logging.info(f"Journey: {journey_id, journey_version+1} CREATED SUCCESSFULLY!")\n        # Stop old version\n        self.change_status(journey_id, journey_version, "stop")\n        logging.info(f"Journey: {journey_id, journey_version} STOPPED SUCCESSFULLY!")\n        '

# Update Journey Activities with New Weights

In [113]:
# Add condition: Do weights add up to 100%.
def filter_weights_not_adding_up_to_100_perc(journey_new_weights):
    
    # Column "split" should be present, as it represents the A/B Test name to be grouped by
    journey_new_weights["total_weights"] = journey_new_weights.groupby(["journey_name", "split"]).new_weight.transform(np.sum)
    
    # Get path_names / email_names that do not match this criterion for log purposes
    email_names_not_adding_up_to_100_perc = journey_new_weights.loc[journey_new_weights["total_weights"] != 100]["email_name"].to_list()
    logging.debug(f"Email Names {email_names_not_adding_up_to_100_perc} NOT ADDING UP TO 100%.")
     
    # Filter dataframe keeping only weights adding up to 100% within their respective A/B Test
    journey_new_weights = journey_new_weights.loc[journey_new_weights["total_weights"] == 100]
    
    return journey_new_weights

# get activity key when activity name = email_name
# get activity key when activity outcome next = previous activity key, if does not contain "SPLIT" repeat, else update

def get_activity_key_by_name(journey_activities, email_name):
    
    return [activity["key"] for activity in journey_activities["activities"] if activity["name"] == email_name][0]

def get_activity_key_by_outcome_next(journey_activities, outcome_next):
    
    for activity in journey_activities["activities"]:
    
        for outcome in activity["outcomes"]:

            if ("next" in outcome and outcome["next"] == outcome_next):

                return activity["key"]

def update_weights_for_outcome_next(journey_activities, previous_activity_key, outcome_next, new_weight):
    
    for activity in journey_activities["activities"]:
        
        if activity["key"] == previous_activity_key:
            
            for outcome in activity["outcomes"]:
                
                if ("next" in outcome and outcome["next"] == outcome_next):
                    
                    outcome["arguments"]["percentage"] = str(new_weight)
                    
                    outcome["metaData"]["label"] = str(new_weight) + "%"
    
    return journey_activities
    
def update_journey_activities(journey_activities, journey_new_weights):
    
    for email_name in journey_new_weights["email_name"]:  # "EN_A"
        
        new_weight = journey_new_weights.loc[journey_new_weights["email_name"] == email_name, "new_weight"].values[0]
    
        outcome_next = get_activity_key_by_name(journey_activities, email_name)  # "EMAILV2-3"

        previous_activity_key = get_activity_key_by_outcome_next(journey_activities, outcome_next)  # "STOWAIT-1"

        while "SPLIT" not in previous_activity_key:

            outcome_next = previous_activity_key  # "STOWAIT-1"

            previous_activity_key = get_activity_key_by_outcome_next(journey_activities, outcome_next)  # 'RANDOMSPLITV2-1'
            
        update_weights_for_outcome_next(journey_activities, previous_activity_key, outcome_next, new_weight)
        
    return journey_activities

# Reading Data

In [114]:
# Read journey new weights - "AbandonedCart Program"
new_weights = pd.read_csv("journey_builder_api_test_prod.csv", sep=";")
new_weights

,journey_name,email_name,new_weight
0,Test_Quentin_Update_Weights_Journey,EN_VA,50
1,Test_Quentin_Update_Weights_Journey,EN_VB,50
2,Test_Quentin_Update_Weights_Journey,ES_VA,50
3,Test_Quentin_Update_Weights_Journey,ES_VB,50
4,Test_Quentin_Update_Weights_Journey,Rem_VA,50
5,Test_Quentin_Update_Weights_Journey,Rem_VB,50


# Main Function

In [117]:
def main_function(journey_name):
    
    print(f"{journey_name} Starting.. Process ID {os.getpid()} CurrentProcess {current_process().name}")
    
    # Get journey new weights from file
    journey_new_weights = new_weights.loc[new_weights["journey_name"] == journey_name].reset_index()
    
    print(journey_name, type(journey_name))
            
    # Instantiate journey object
    journey_object = JourneyBuilder(journey_name)
    
    print(journey_object.journey_name)
    
    # Get journey activities
    journey_activities = journey_object.get_activities()
        
    # Check if status in ("Published", "Draft", "Paused"): log
    journey_id, journey_version, journey_status = journey_object.get_metadata("id", "version", "status")
        
    # If status == "Published" then Pause / Update / sleep(300) / Resume: log
    # Journey cannot be updated if status == "Published"
    # Journey cannot be resumed if status != "Paused"
    # Takes 5' approx to pause a journey
    if journey_status in ["Published"]:
        
        print(f"First cond {journey_name}")

        print(f"{journey_name} First cond: Published..")

        # Pause journey
        journey_object.change_status("Pause")
        logging.debug(f"Journey {journey_id}, Version {journey_version}, Status {journey_status}, PAUSED SUCCESSFULLY")

        print(f"{journey_name} Paused Success..")
        
        # Make sure status is "Paused" before Updating & Resuming (for status consistency within journey activities)
        # Check status every 30"
        while journey_status != "Paused":
            sleep(30)
            journey_status = journey_object.get_metadata("status")[0]
        # log timeout TODO

        # Get journey activities, again because status cannot be "Published" in activities updated
        journey_activities = journey_object.get_activities()
        # Update journey activities
        journey_activities_updated = update_journey_activities(journey_activities, journey_new_weights)
        # Update journey version in API
        journey_object.update_version(journey_activities_updated)
        logging.debug(f"Journey {journey_id}, Version {journey_version}, Status {journey_status}, UPDATED SUCCESSFULLY")
        print(f"{journey_name} Updated Success..")

        # Raise an error if status != "Paused"
        # journey_status = journey_object.get_metadata("status")[0]
        # if journey_status != "Paused":
        #    raise StatusError(
        #       journey_status, f"status = {journey_status}, should be Paused.") 
        
        # Resume journey
        journey_object.change_status("Resume")
        logging.debug(f"Journey {journey_id}, Version {journey_version}, Status {journey_status}, RESUMED SUCCESSFULLY")
        print(f"{journey_name} Resumed Success..")

    # If status in ("Paused", "Draft") then Update / Do not publish: log
    elif journey_status in ["Paused", "Draft"]:
        
        print(f"First cond {journey_name}")

        print(f"{journey_name} First cond: Paused/Draft..")

        # Update journey activities
        journey_activities_updated = update_journey_activities(journey_activities, journey_new_weights)
        # Update journey version in API
        journey_object.update_version(journey_activities_updated)
        logging.debug(f"Journey {journey_id}, Version {journey_version}, Status {journey_status}, UPDATED SUCCESSFULLY")
        print(f"{journey_name} Updated Success..")

    else:
        logging.debug(f"Journey {journey_id}, Version {journey_version}, Status {journey_status}, IGNORED.")

In [12]:
start = time.perf_counter()

processes = []

for journey_name in new_weights["journey_name"].unique():
    p = Process(target=main_function, args=(journey_name,))
    p.start()
    processes.append(p)

# Wait for each process to finish before moving on to next code
for process in processes:
    process.join()
    
end = time.perf_counter()

print(f"total time: {end-start}")

total time: 0.2416249999999991


In [118]:
for journey_name in new_weights["journey_name"].unique():
    main_function(journey_name)

Test_Quentin_Update_Weights_Journey Starting.. Process ID 9996 CurrentProcess MainProcess
Test_Quentin_Update_Weights_Journey <class 'str'>
Test_Quentin_Update_Weights_Journey
First cond Test_Quentin_Update_Weights_Journey
Test_Quentin_Update_Weights_Journey First cond: Published..
Test_Quentin_Update_Weights_Journey Paused Success..
Test_Quentin_Update_Weights_Journey Updated Success..
Test_Quentin_Update_Weights_Journey Resumed Success..


In [104]:
j = JourneyBuilder("Test_Quentin_Update_Weights_Journey")

In [105]:
activities = j.get_activities()
activities

{'id': '09756e24-2ae1-4351-888b-4a8504d2a7c2',
 'key': 'fddfeee5-f2b7-58c2-cb3a-0390558d92f7',
 'name': 'Test_Quentin_Update_Weights_Journey',
 'lastPublishedDate': '2021-11-15T09:45:36',
 'description': '',
 'version': 1,
 'workflowApiVersion': 1.0,
 'createdDate': '2021-11-15T09:42:38.747',
 'modifiedDate': '2021-11-16T04:36:23.14',
 'activities': [{'id': '5efa6db1-049e-4a39-9631-40e3ee40583b',
   'key': 'MULTICRITERIADECISIONV2-1',
   'name': '',
   'description': '',
   'type': 'MULTICRITERIADECISION',
   'outcomes': [{'key': 'default_path_1',
     'next': 'RANDOMSPLITV2-1',
     'arguments': {},
     'metaData': {'label': 'Lang=EN',
      'skipI18n': True,
      'isLabelFromConversion': False,
      'criteriaDescription': 'Contacts ID equal 0',
      'invalid': False}},
    {'key': '422c652d-cc7d-a779-17ba-7261d3a045f8',
     'next': 'RANDOMSPLITV2-2',
     'arguments': {},
     'metaData': {'label': 'Lang=ES',
      'skipI18n': True,
      'isLabelFromConversion': False,
      'c